In [6]:
import pandas as pd
import os
import altair as alt
import geopandas as gpd
import geobr
import panel as pn

In [7]:
alt.data_transformers.disable_max_rows()
alt.renderers.enable('default')
pn.extension('vega')

## Carrega dados
### Dados da Covid-19

In [8]:
diretorio_covid = os.fsencode('dados/covid')
df_covid_base = None
for arquivo in os.listdir(diretorio_covid):
    nome_arquivo = 'dados/covid/' + os.fsdecode(arquivo)
    df_temp = pd.read_csv(nome_arquivo, decimal=',', sep=';')
    if df_covid_base is None:
        df_covid_base = df_temp
    else:
        df_covid_base = pd.concat([df_covid_base, df_temp])

df_covid_base.data = pd.to_datetime(df_covid_base.data, infer_datetime_format=True)
df_covid_base = df_covid_base.sort_values('data')

In [9]:
df_covid_base

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-25,9,210147125.0,0.0,0,0,0,0.0,0.0,NaN
2615,Nordeste,BA,NaN,29,NaN,NaN,NaN,2020-02-25,9,14873064.0,0.0,0,0,0,NaN,NaN,NaN
2457,Nordeste,SE,NaN,28,NaN,NaN,NaN,2020-02-25,9,2298696.0,0.0,0,0,0,NaN,NaN,NaN
2299,Nordeste,AL,NaN,27,NaN,NaN,NaN,2020-02-25,9,3337357.0,0.0,0,0,0,NaN,NaN,NaN
2141,Nordeste,PE,NaN,26,NaN,NaN,NaN,2020-02-25,9,9557071.0,0.0,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352641,Sudeste,MG,São Sebastião da Bela Vista,31,316440.0,31007.0,POUSO ALEGRE,2022-04-26,17,5504.0,905.0,0,16,0,NaN,NaN,0.0
441613,Sudeste,SP,Riolândia,35,354420.0,35157.0,VOTUPORANGA,2022-04-26,17,12518.0,2468.0,0,32,0,NaN,NaN,0.0
648557,Centro-Oeste,GO,São Luiz do Norte,52,522015.0,52013.0,SAO PATRICIO I,2022-04-26,17,5167.0,942.0,0,20,0,NaN,NaN,0.0
515853,Sul,SC,Forquilhinha,42,420545.0,42015.0,CARBONIFERA,2022-04-26,17,26793.0,8197.0,1,64,0,NaN,NaN,0.0


In [10]:
df_municipios = df_covid_base[df_covid_base.municipio.isnull() == False].copy()
df_municipios['codmun']  = df_municipios.codmun.astype('Int64').astype(str)
df_brasil = df_municipios.groupby(['data', 'semanaEpi']).sum()[['populacaoTCU2019', 'casosAcumulado', 'casosNovos', 'obitosAcumulado', 'obitosNovos']].reset_index()
df_regioes = df_municipios.groupby(['regiao', 'data', 'semanaEpi']).sum()[['populacaoTCU2019', 'casosAcumulado', 'casosNovos', 'obitosAcumulado', 'obitosNovos']].reset_index()

In [11]:
df_municipios

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
304993,Sudeste,MG,Caiana,31,311010,31042.0,CARANGOLA,2020-03-27,13,5496.0,0.0,0,0,0,NaN,NaN,0.0
143830,Nordeste,RN,Barcelona,24,240150,24005.0,5ª REGIAO DE SAUDE - SANTA CRUZ,2020-03-27,13,3998.0,0.0,0,0,0,NaN,NaN,0.0
658815,Centro-Oeste,MS,Douradina,50,500350,50003.0,DOURADOS,2020-03-27,13,5924.0,0.0,0,0,0,NaN,NaN,0.0
358460,Sudeste,MG,Olímpio Noronha,31,314550,31008.0,SAO LOURENCO,2020-03-27,13,2787.0,0.0,0,0,0,NaN,NaN,0.0
20132,Norte,AM,Rio Preto da Eva,13,130356,13001.0,"MANAUS, ENTORNO E ALTO RIO NEGRO",2020-03-27,13,33347.0,0.0,0,0,0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352641,Sudeste,MG,São Sebastião da Bela Vista,31,316440,31007.0,POUSO ALEGRE,2022-04-26,17,5504.0,905.0,0,16,0,NaN,NaN,0.0
441613,Sudeste,SP,Riolândia,35,354420,35157.0,VOTUPORANGA,2022-04-26,17,12518.0,2468.0,0,32,0,NaN,NaN,0.0
648557,Centro-Oeste,GO,São Luiz do Norte,52,522015,52013.0,SAO PATRICIO I,2022-04-26,17,5167.0,942.0,0,20,0,NaN,NaN,0.0
515853,Sul,SC,Forquilhinha,42,420545,42015.0,CARBONIFERA,2022-04-26,17,26793.0,8197.0,1,64,0,NaN,NaN,0.0


### Dados geográficos

In [12]:
try:
    nome_arquivo = "dados/mapa_municipios.geojson"
    mapa_municipios = gpd.read_file(nome_arquivo, driver='GeoJSON')
except:
    mapa_municipios = geobr.read_municipality(year=2020)
    mapa_municipios['code_muni']  = mapa_municipios.code_muni.astype('Int64').astype(str)
    mapa_municipios['code_muni_6'] = mapa_municipios.code_muni.apply(lambda code_muni: code_muni[:6])
    mapa_municipios.to_file("dados/mapa_municipios.geojson", driver='GeoJSON')
mapa_municipios.head(5)

/home/helder/Codigos/Python/VisualizacaoDadosCovid/venv/lib/python3.8/site-packages/geopandas/array.py:93: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


,code_muni,name_muni,code_state,abbrev_state,name_state,code_region,name_region,code_muni_6,geometry
0,1100015,Alta Floresta D'oeste,11.0,RO,Rondônia,1.0,Norte,110001,"MULTIPOLYGON (((-62.19465 -11.82746, -62.18945..."
1,1100023,Ariquemes,11.0,RO,Rondônia,1.0,Norte,110002,"MULTIPOLYGON (((-62.53648 -9.73222, -62.52765 ..."
2,1100031,Cabixi,11.0,RO,Rondônia,1.0,Norte,110003,"MULTIPOLYGON (((-60.37119 -13.36655, -60.37661..."
3,1100049,Cacoal,11.0,RO,Rondônia,1.0,Norte,110004,"MULTIPOLYGON (((-61.00080 -11.29737, -61.00103..."
4,1100056,Cerejeiras,11.0,RO,Rondônia,1.0,Norte,110005,"MULTIPOLYGON (((-61.49976 -13.00525, -61.49426..."


## Cria gráficos

### Série de óbitos no Brasil

In [13]:
df_brasil['obitosNovosMedia14Dias'] = round(df_brasil[['obitosNovos']].rolling(window=14).mean(), 2)

In [14]:
figura_serie_novos_obitos = alt.Chart(df_brasil).mark_line().encode(
    x='data',
    y='obitosNovosMedia14Dias',
    tooltip=['data','obitosNovosMedia14Dias']
).interactive()

### Série do acumulado de óbitos por região

In [15]:
df_regioes['taxaObitosAcumulado'] = round(df_regioes.obitosAcumulado / df_regioes.populacaoTCU2019 * 100000, 2)

In [16]:
figura_serie_obitos_acumulados = alt.Chart(df_regioes).mark_line().encode(
    x='data',
    y='taxaObitosAcumulado',
    color='regiao',
    tooltip=['regiao', 'data','taxaObitosAcumulado']
).interactive()

## Dashboard

In [17]:
data_minima = df_covid_base.data.min()
data_maxima = df_covid_base.data.max()

In [29]:
titulo = 'Mortalidade por Covid-19 no Brasil'

def get_mapa():
    if data_maxima == date_range_slider.value[1]:
        data_maxima_selecionada = date_range_slider.value[1]
    else:
        data_maxima_selecionada = date_range_slider.value[1].replace(hour=0, minute=0, second=0)

    df_taxa_mortalidade = df_municipios[df_municipios.data == data_maxima_selecionada].set_index('codmun').copy()

    df_taxa_mortalidade['Taxa de mortalidade'] = round(df_taxa_mortalidade.obitosAcumulado / df_taxa_mortalidade.populacaoTCU2019 * 100000, 2)

    geodf_municipios = mapa_municipios.merge(df_taxa_mortalidade, left_on='code_muni_6', right_on='codmun').set_index('municipio')

    limiar_outlier_superior = geodf_municipios['Taxa de mortalidade'].describe()['75%'] + 2 * (geodf_municipios['Taxa de mortalidade'].describe()['75%'] - geodf_municipios['Taxa de mortalidade'].describe()['25%'])
    geodf_municipios = geodf_municipios[geodf_municipios['Taxa de mortalidade'] <= limiar_outlier_superior]

    limiar_outlier_inferior = 0
    geodf_municipios = geodf_municipios[geodf_municipios['Taxa de mortalidade'] >= limiar_outlier_inferior]

    figura_mapa_mortalidade = alt.Chart(geodf_municipios).mark_geoshape().encode(color='Taxa de mortalidade:Q',tooltip=['name_muni','Taxa de mortalidade'])

    return figura_mapa_mortalidade

date_range_slider = pn.widgets.DateRangeSlider(
    name='Período',
    start=data_minima, end=data_maxima,
    value=(data_minima, data_maxima))

button = pn.widgets.Button(name='Atualizar', button_type='primary')

coluna1 = pn.Column("### Novos óbitos", figura_serie_novos_obitos, "### Mortalidade nos municípios", get_mapa)
coluna2 = pn.Column("### Óbitos acumulados", figura_serie_obitos_acumulados)

def click_botao(event):
    coluna1[3] = get_mapa()

button.on_click(click_botao)

dashboard = pn.Row(
    coluna1,
    coluna2
)

In [30]:
pn.template.FastListTemplate(
    title=titulo,
    sidebar=[date_range_slider, button],
    main=[dashboard]
).servable();